# extract text and its position

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO


def convert_pdf_to_txt(path1, path2, num = 0):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path1, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for i, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
        if num > 0 and i > num:
            break
        interpreter.process_page(page)

    text = retstr.getvalue()
    file_name = 'page' + str(num)
    f = open(path2+file_name, 'w')
    f.write(text)
    f.close()
    #text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()




# set paths

In [2]:
#from Tkinter import *
#import tkFileDialog
from tkinter import *
import tkinter.filedialog

win = Tk()
win.title('Select the PDF file')
var = StringVar()

w = Label(win, text="File Path:")
e = Entry(win, textvariable=var)
#b = Button(win, text="Browse",command=lambda:var.set(tkFileDialog.askopenfilename()))
b = Button(win, text="Browse",command=lambda:var.set(tkinter.filedialog.askopenfilename()))
w.pack(side=LEFT)
e.pack(side=LEFT)
b.pack(side=LEFT)
win.mainloop()

path1 = var.get()

win = Tk()
win.title('Select the output file\'s direnctory')
var = StringVar()

w = Label(win, text="File Path:")
e = Entry(win, textvariable=var)
#b = Button(win, text="Browse",command=lambda:var.set(tkFileDialog.askdirectory()))
b = Button(win, text="Browse",command=lambda:var.set(tkinter.filedialog.askdirectory()))
w.pack(side=LEFT)
e.pack(side=LEFT)
b.pack(side=LEFT)
win.mainloop()

path2 = var.get()+'/'

In [3]:
#path2 = 'C:/Users/zhongzhu/Documents/GitHub/work/temp/'
#path1 = 'C:/Users/zhongzhu/Documents/GitHub/work/Seattle.pdf'
#print(path1, path2)

In [4]:
convert_pdf_to_txt(path1, path2)

# record positions

In [4]:
import re

def record_pos(path, page=0, LINE_HEIGHT = 1, start_page=0, end_page=None):
    f = open(path+'page'+str(page), 'r')
    pages = f.read().split('#PAGE')
    f.close()
    
    contents = {}
    for page in pages[start_page: end_page]:
        page_num = page[page.find('<<<')+3:page.find('>>>')]
        page_, num = re.subn(r'<<<.*?>>>', '',page)
    
        boxes = page_.split('TEXT_BOX')

        for content in boxes:
            c = content.split(';')
            if(c[0].find('left:') > 0): #if not empty
                l = c[0][c[0].find('left:')+5:c[0].find('px')]
                t = c[1][c[1].find('top:')+4:c[1].find('px')]
                w = c[2][c[2].find('width:')+6:c[2].find('px')]
                h = c[3][c[3].find('height:')+7:c[3].find('px')]

                text = "".join(c[4:])
                #contents[text.decode('utf-8')] = [(int(l), int(t), int(w), int(h))] #whole box
                lines = text.split('\n')

                tempCount = 0
                for line in lines:
                    if re.search(r'[A-Za-z0-9,.?!]', line):
                        if not line in contents:
                            contents[line] = [(int(l), int(t)+tempCount, int(w), int(h), page_num)]
                        else:
                            contents[line].append((int(l), int(t)+tempCount, int(w), int(h), page_num))
                    tempCount += LINE_HEIGHT ### default value 1
    return contents


In [5]:
contents = record_pos(path2)
#for pos in contents:
    #print(pos)
    #print(contents[pos])

In [6]:
#import Tkinter
#import tkSimpleDialog
import tkinter
import tkinter.simpledialog

#root = Tkinter.Tk()
root = tkinter.Tk()
root.title('select sample pages')
var1 = tkinter.StringVar()
var2 = tkinter.StringVar()
#b1 = Button(root, text="Select start page",command=lambda:var1.set(tkSimpleDialog.askinteger("Select start page", "enter page number", parent = root)))
#b2 = Button(root, text="Select end page",command=lambda:var2.set(tkSimpleDialog.askinteger("Select end page", "enter page number", parent = root)))
b1 = tkinter.Button(root, text="Select start page",command=lambda:var1.set(tkinter.simpledialog.askinteger("Select start page", "enter page number", parent = root)))
b2 = tkinter.Button(root, text="Select end page",command=lambda:var2.set(tkinter.simpledialog.askinteger("Select end page", "enter page number", parent = root)))
b1.grid(column = 0, row = 0)
b2.grid(column = 1, row = 0)
#var = tkSimpleDialog.askstring("Select Pates", "enter your name", parent = root)
root.mainloop()
start_page = var1.get()
end_page = var2.get()

In [7]:
# get the positions of some sample pages
first_page_contents = record_pos(path2, start_page=int(start_page), end_page=int(end_page))

In [8]:
#for i in first_page_contents:
    #print(i)
    #print(first_page_contents[i])

In [9]:
filted_contents = {}
for text in contents:
    if len(contents[text]) > 60:  ###set at beginning#####################
        filted_contents[text] = contents[text]
        #print(text + ' ' +str(len(filted_contents[text])))

In [10]:
#from Tkinter import *
#import Tkinter as tk
import tkinter as tk

class Example(tk.Frame):
     
    def __init__(self, root, *args, **kwargs):
        tk.Frame.__init__(self, root, *args, **kwargs)
        self.root = root
        self.vsb = tk.Scrollbar(self, orient="vertical")
        self.text = tk.Text(self, width=40, height=20, yscrollcommand=self.vsb.set)
        self.vsb.config(command=self.text.yview)
        self.vsb.pack(side="right", fill="y")
        self.text.pack(side="left", fill="both", expand=True)
        
        self.temp_contents = {}

        var = []
        c = []
        for i, cat in enumerate(sorted(filted_contents)):
            c.append(cat)
            v = tk.BooleanVar()
            var.append(v)
            cb = tk.Checkbutton(self, text="#%s" % cat, variable=var[i])
            self.text.window_create("end", window=cb)
            self.text.insert("end", "\n") # to force one checkbox per line
                      
        def var_states():
            for i,v in enumerate(var):
                if v.get():
                    #print(c[i])
                    self.temp_contents[c[i]] = filted_contents[c[i]]
        tk.Button(root, text='Select', command=var_states).pack()
        
root = tk.Tk()
root.title('selecte categories')
win = Example(root)
win.pack(side="top", fill="both", expand=True)
root.mainloop()

filted_contents = win.temp_contents

# search neighbors

In [11]:
from itertools import product
neighbors = {}
    
for line1, line2 in product(filted_contents, first_page_contents):
    key = line1
    value = line2
    if not key in neighbors:
        neighbors[key] = []
    if(line1 != line2 and line1 in first_page_contents
       and abs(int(first_page_contents[line1][0][0])-int(first_page_contents[line2][0][0]))<1000    ###set at beginning
       and abs(int(first_page_contents[line1][0][1])-int(first_page_contents[line2][0][1]))<100): ###set at beginning
        neighbors[key].append(value)

In [12]:
#for key in neighbors:
    #print(key, neighbors[key])

# GUI

In [13]:
#import Tkinter as tk
#import ttk
import tkinter as tk
import tkinter.ttk

win = tk.Tk()
win.title("Set Pairs")   

#ttk.Label(win, text="Select category:").grid(column=0, row=0)   
tkinter.ttk.Label(win, text="Select category:").grid(column=0, row=0) 
#ttk.Label(win, text="content starts from").grid(column=2, row=0) 
#ttk.Label(win, text="content ends before").grid(column=3, row=0)     
pos_or_area = StringVar()
def select():  
    if pos_or_area.get() == 'area':
        ls = list(neighbors.keys())   
        ls += ['#END']
        dropdown2['values'] = ls
    else:
        ls = neighbors[category.get()]
        dropdown2['values'] = ls
    dropdown2.current()    

R1 = tkinter.ttk.Radiobutton(win, text="find by position", variable=pos_or_area, value='pos', command=select)
R1.grid(column = 2, row = 0)
R2 = tkinter.ttk.Radiobutton(win, text="find by area", variable=pos_or_area, value='area', command=select)
R2.grid(column = 3, row = 0)
    
pairs = {}
def bind():   
    #key = contents[category.get()]
    #new_value = contents[content.get()]
    key = category.get()
    if not key in pairs:
        pairs[key] = set([])
    
    if pos_or_area.get()=='pos':
        a = first_page_contents[category.get()][0]
        b = first_page_contents[content.get()][0]
        c = (b[0]-a[0], b[1]-a[1], b[2]-a[2], b[3]-a[3], b[4], pos_or_area.get())
        pairs[key].add(c)
    else:
        c = (content.get(), pos_or_area.get()) 
        pairs[key].add(c)
        
    dropdown2.current()
    
# creat a drop-down list
category = tk.StringVar()
dropdown1 = tkinter.ttk.Combobox(win, width=40, height=20, textvariable=category)
dropdown1['values'] = list(neighbors.keys())   
dropdown1.grid(column=0, row=1)      
#dropdown1.current(0)     


# create another drop-down list
content = tk.StringVar()
dropdown2 = tkinter.ttk.Combobox(win, width=40, textvariable=content)    
dropdown2.grid(column=2, row=1)  
 

# bind button
action = tkinter.ttk.Button(win, text="Bind", command=bind)
action.grid(column=4, row=1) 

win.mainloop()      # window

In [14]:
#for i in pairs:
    #print(i)
    #print(type(pairs[i]))
    #print((pairs[i]))
    #for i in pairs[i]:
        #print(len(i))

In [15]:
import re

page = 0
f = open(path2+'page'+str(page), 'r')
text = f.read()
text, num = re.subn(r'TEXT_BOX.*?height:[\d]+px;', '',text)
f.close()

reports = {}
pages = text.split('#PAGE')
for page in pages:
    if(len(page) == 0):
        continue
    page_num = page[page.find('<<<')+3:page.find('>>>')]
    page_, num = re.subn(r'<<<.*?>>>', '',page)
    reports[int(page_num)] = page_
    

In [16]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def pdf2txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [17]:
import re

text = pdf2txt(path1)

reports = {}
pages = text.split('#PAGE')
for page in pages:
    if(len(page) == 0):
        continue
    page_num = page[page.find('<<<')+3:page.find('>>>')]
    page_, num = re.subn(r'<<<.*?>>>', '',page)
    try:
        reports[int(page_num)] = page_
    except:
        print(page)

In [18]:
def find_content_by_area(text, start, end, head='CONTRACT NUMBER'): #####
    #p = re.compile(r'{%s}.*{%s}' % (start, end))
    start = start.lstrip()
    end = end.lstrip()
    content= []
    flag = False
    #for page_num, page in sorted(reports.iteritems()):
    for page_num, page in sorted(reports.items()):
        search = []
        if flag:
            flag = False
            search = re.findall('%s.*?%s' % (head, end), page, re.DOTALL)
        elif end != 'END' and start in page and end in page:
            search = re.findall('%s.*?%s' % (start, end), page, re.DOTALL)
        else:
            search = re.findall('%s.*' % start, page, re.DOTALL)
            flag = True
        for i in search:
            i = i.replace(start, '')
            i = i.replace(end, '')
            i = i.replace('\n\n', ' ')
            #content.append('#PAGE_'+str(page_num)+' '+i)
            content.append(' ' + i)
    #print(content)
    return content

In [19]:
data = {}
for cat in pairs:
    for c in pairs[cat]:
        if c[-1] == 'area':
            content = find_content_by_area(text, cat, c[0], )
            data[cat] = content

In [20]:
def find_content_by_pos(content_map, content_pos):
    content = ''
    for c in content_map:
        m = 1 #### maximal error allowed#########################
        for pos in content_map[c]:
            error = abs(content_pos[0]-pos[0]) + abs(content_pos[1]-pos[1])
            #print error
            if error < m:
                content += '#PAGE_'+pos[-1]+' '+c
    #print(m)
    return content
    
#data = {}
content_map = record_pos(path2, 0)
for cat in pairs:
    print(pairs[cat])
    #if cat in content_map:  #unnecessary??
    for pos in content_map[cat]:
        content = ''
        for shift in pairs[cat]:
            if shift[-1] == 'pos':
                content_pos = (pos[0]+shift[0], pos[1]+shift[1], pos[2]+shift[2], pos[3]+shift[3])
                content += find_content_by_pos(content_map, content_pos)
        if len(content) != 0:
            if cat in data: #unnecessary??
                data[cat].append(content)
            else:
                data[cat] = [content]
           

{(' EQUIPMENT HOURS', 'area')}


# generate CSV table

In [30]:
#for i in data:
    #print (i)
    #for j in data[i]:
        #print (j)
#print(len(data))

In [21]:
import csv
f = open(path2+'result0927_py3.csv', 'w', newline = '')
writer = csv.writer(f)

for i, cat in enumerate(data):
    #if i >50:
        #break
    writer.writerow([cat] + data[cat])
    #print(i) 
    #print(cat)
f.close()

# new 

In [21]:
ehab_stop_words = []

for word in open(r"ehabs_stop_words.txt",'r'):
    ehab_stop_words.append(word.strip())

from sklearn.feature_extraction import text 

my_stop_words = text.ENGLISH_STOP_WORDS.union(ehab_stop_words) # union the ehab's stop words and the stop words in the pachage

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
import csv
import numpy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

count_vect = CountVectorizer(stop_words=my_stop_words, analyzer='word')
tfidf = TfidfVectorizer(stop_words=my_stop_words, analyzer='word') 
X_counts = {}


with open('words_counts.csv', 'w', newline = '') as f:
    for i, cat in enumerate(data):
        X_counts[cat] = count_vect.fit_transform(data[cat]) # counting the words in every reports
        inv_dic = {v: k for k, v in count_vect.vocabulary_.items()}
            
        X_tfidf=tfidf.fit_transform(data[cat])
        kmeans_word_tfidf = KMeans(n_clusters=5, init='random').fit(X_tfidf.transpose()) #clustering words

        writer = csv.writer(f)
        counts_toarray = X_counts[cat].transpose().toarray()

        writer.writerow([cat])
        writer.writerow(['word_number', 'word_label', 'word'])
        print(counts_toarray.shape) 
        for counter in range(0,len(counts_toarray)):
            try:
                if not re.match(r'.*[\d]+.*', inv_dic[counter]): #exclude the words containing numbers
                    counts_row = counts_toarray[counter].ravel()
                    writer.writerow(numpy.append([counter, kmeans_word_tfidf.labels_[counter], inv_dic[counter]], counts_row))
            except:
                print(counter)

f.close()

(1250, 881)
(3412, 747)
(2499, 862)
(27, 497)
(1202, 270)
(1016, 391)
(3001, 860)
(946, 555)
(2659, 869)
(29, 89)
(2819, 840)
(2031, 2027)
(884, 881)
(2032, 2027)
(2485, 1747)
(999, 881)
(1800, 1747)


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
import csv
import numpy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

count_vect = CountVectorizer(stop_words=my_stop_words, analyzer='word',ngram_range=(1, 2))
tfidf = TfidfVectorizer(stop_words=my_stop_words, analyzer='word',ngram_range=(1, 2)) 
X_counts = {}


with open('words_tfidf_ngram2.csv', 'w', newline = '') as f:
    for i, cat in enumerate(data):
        X_tfidf=tfidf.fit_transform(data[cat]) # counting the words in every reports
        inv_dic = {v: k for k, v in tfidf.vocabulary_.items()}
            
        
        kmeans_word_tfidf = KMeans(n_clusters=5, init='random').fit(X_tfidf.transpose()) #clustering words

        writer = csv.writer(f)
        tfidf_toarray = X_tfidf.transpose().toarray()
        writer.writerow([cat])
        writer.writerow(['word_number', 'word_label', 'word'])
        print(tfidf_toarray.shape) 
        for counter in range(0,len(tfidf_toarray)):
            try:
                if not re.match(r'.*[\d]+.*', inv_dic[counter]): #exclude the words containing numbers
                    tfidf_row = tfidf_toarray[counter].ravel()
                    writer.writerow(numpy.append([counter, kmeans_word_tfidf.labels_[counter], inv_dic[counter]], tfidf_row))
            except:
                print(counter)

f.close()



(4128, 262)


In [23]:
print(inv_dic)

{1123: 'contract', 2232: 'number', 1186: 'daca67', 14: '02', 107: '2012', 2: '0004', 4094: 'weather', 1043: 'caused', 1208: 'delay', 3869: 'temperature', 2133: 'min', 213: '32', 2053: 'max', 462: '50', 3359: 'precipitation', 64: '14', 2193: 'mph', 4102: 'wind', 1128: 'contractors', 3499: 'quality', 1134: 'control', 3584: 'report', 3497: 'qcr', 1942: 'log', 1113: 'construction', 2130: 'military', 3434: 'project', 3579: 'replace', 1416: 'family', 1638: 'housing', 2336: 'p5', 1125: 'contractor', 1395: 'evergreene', 3489: 'qc', 2202: 'narratives', 798: 'activities', 3418: 'progress', 791: 'aai', 3741: 'shutting', 1264: 'dow', 896: 'ater', 3555: 'removing', 3515: 'refrigerators', 1548: 'general', 1098: 'comments', 1735: 'inspections', 3598: 'results', 3912: 'today', 3880: 'testing', 3523: 'remarks', 3688: 'safety', 1726: 'inspection', 2113: 'meetings', 1814: 'issues', 3405: 'preparatory', 1670: 'initials', 1608: 'held', 1198: 'day', 2250: 'onsite', 3398: 'prep', 1667: 'initial', 1194: 'date

In [77]:

for i, cat in enumerate(data):
    X_counts[cat] = count_vect.fit_transform(data[cat]) # counting the words in every reports
    inv_dic = {v: k for k, v in count_vect.vocabulary_.items()}

    for cat in X_counts:
        print(cat)
        X_tfidf=tfidf.fit_transform(data[cat])
        kmeans_word_tfidf = KMeans(n_clusters=5, init='random').fit(X_tfidf.transpose()) #clustering words

        counts_toarray = X_counts[cat].transpose().toarray()
        counter = 0
        for word_num in range(0,1799):
            if not re.match(r'.*[\d]+.*', inv_dic[counter]): #exclude the words containing numbers
                counts_row = counts_toarray[word_num].ravel()
                print(counter, kmeans_word_tfidf.labels_[counter], inv_dic[counter], counts_row)
            counter += 1
    print(X_counts[cat].shape) 


[658, 0, 'accidents'] [0 0 0 ..., 0 2 2]
[659, 4, 'activities'] [0 0 0 ..., 0 0 0]
[660, 4, 'activity'] [0 0 0 ..., 0 0 0]
[661, 4, 'add'] [0 0 0 ..., 0 0 0]
[662, 4, 'advance'] [0 0 0 ..., 0 0 0]
[663, 4, 'aldave'] [0 0 0 ..., 0 0 0]
[664, 4, 'alking'] [0 0 0 ..., 0 0 0]
[665, 4, 'alls'] [0 0 0 ..., 0 0 0]
[666, 4, 'american'] [0 0 0 ..., 0 0 0]
[667, 4, 'amp'] [0 0 0 ..., 0 0 0]
[668, 4, 'analysis'] [0 0 0 ..., 0 0 0]
[669, 4, 'appliances'] [0 0 0 ..., 0 0 0]
[670, 4, 'apprentice'] [0 0 0 ..., 0 0 0]
[671, 4, 'approval'] [0 0 0 ..., 0 0 0]
[672, 4, 'apr'] [0 0 0 ..., 0 0 0]
[673, 4, 'arauco'] [0 0 0 ..., 0 0 0]
[674, 4, 'architect'] [0 0 0 ..., 0 0 0]
[675, 4, 'area'] [0 0 0 ..., 0 0 0]
[676, 4, 'arned'] [0 0 0 ..., 0 0 0]
[677, 4, 'arts'] [0 0 0 ..., 0 0 0]
[678, 4, 'ash'] [0 0 0 ..., 0 0 0]
[679, 4, 'assistant'] [0 0 0 ..., 0 0 0]
[680, 4, 'associates'] [0 0 0 ..., 0 0 0]
[681, 4, 'ater'] [0 0 0 ..., 0 0 0]
[682, 4, 'attached'] [0 0 0 ..., 0 0 0]
[683, 4, 'attended'] [0 0 0 ..., 0 

[675, 3, 'area'] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0

[693, 2, 'bla'] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 

[698, 1, 'days'] [0 0 0 ..., 0 0 0]
[699, 1, 'debris'] [0 0 0 ..., 0 0 0]
[700, 1, 'decided'] [0 0 0 ..., 0 0 0]
[701, 1, 'decision'] [0 0 0 ..., 0 0 0]
[702, 1, 'decurdment'] [0 0 0 ..., 0 0 0]
[703, 1, 'deep'] [0 0 0 ..., 0 0 0]
[704, 1, 'deficiencies'] [0 0 0 ..., 0 0 0]
[705, 1, 'degree'] [0 0 0 ..., 0 0 0]
[706, 1, 'degrees'] [0 0 0 ..., 0 0 0]
[707, 1, 'delay'] [0 0 0 ..., 0 0 0]
[708, 1, 'delayed'] [0 0 0 ..., 0 0 0]
[709, 1, 'delaying'] [0 0 0 ..., 0 0 0]
[710, 1, 'delays'] [0 0 0 ..., 0 0 0]
[711, 1, 'delete'] [0 0 0 ..., 0 0 0]
[712, 1, 'deliver'] [0 0 0 ..., 0 0 0]
[713, 1, 'delivered'] [0 0 0 ..., 0 0 0]
[714, 1, 'delivery'] [0 0 0 ..., 0 0 0]
[715, 1, 'demo'] [0 0 0 ..., 0 0 0]
[716, 1, 'demobilization'] [0 0 0 ..., 0 0 0]
[717, 1, 'demobilize'] [0 0 0 ..., 0 0 0]
[718, 1, 'demolished'] [0 0 0 ..., 0 0 0]
[719, 1, 'demolition'] [0 0 0 ..., 0 0 0]
[720, 1, 'denied'] [0 0 0 ..., 0 0 0]
[721, 1, 'density'] [0 0 0 ..., 0 0 0]
[722, 1, 'department'] [0 0 0 ..., 0 0 0]
[723, 1, 

[1149, 1, 'issue'] [0 0 0 ..., 0 0 0]
[1150, 1, 'issued'] [0 0 0 ..., 0 0 0]
[1151, 1, 'issues'] [0 0 0 ..., 0 0 0]
[1152, 1, 'itch'] [0 0 0 ..., 0 0 0]
[1153, 1, 'itches'] [0 0 0 ..., 0 0 0]
[1154, 1, 'item'] [0 0 0 ..., 0 0 0]
[1155, 1, 'items'] [0 0 0 ..., 0 0 0]
[1156, 1, 'ith'] [0 0 0 ..., 0 0 0]
[1157, 1, 'ithout'] [0 0 0 ..., 0 0 0]
[1158, 1, 'jack'] [0 0 0 ..., 0 0 0]
[1159, 1, 'jambs'] [0 0 0 ..., 0 0 0]
[1160, 1, 'jams'] [0 0 0 ..., 0 0 0]
[1161, 1, 'jb'] [0 0 0 ..., 0 0 0]
[1162, 1, 'jim'] [0 0 0 ..., 0 0 0]
[1163, 1, 'job'] [0 0 0 ..., 0 0 0]
[1164, 1, 'joe'] [0 0 0 ..., 0 0 0]
[1165, 1, 'jogging'] [0 0 0 ..., 0 0 0]
[1166, 1, 'john'] [0 0 0 ..., 0 0 0]
[1167, 1, 'joints'] [0 0 0 ..., 0 0 0]
[1168, 1, 'joist'] [0 0 0 ..., 0 0 0]
[1169, 1, 'joists'] [0 0 0 ..., 0 0 0]
[1170, 1, 'juggy'] [0 0 0 ..., 0 0 0]
[1171, 1, 'jumps'] [0 0 0 ..., 0 0 0]
[1172, 1, 'keeping'] [0 0 0 ..., 0 0 0]
[1173, 1, 'keys'] [0 0 0 ..., 0 0 0]
[1174, 1, 'kids'] [0 0 0 ..., 0 0 0]
[1175, 1, 'kirby'] [

KeyboardInterrupt: 

In [22]:
f = open(path2+"test", 'w')
f.write(text)
f.close()